# Tabular models

In [ ]:
from fastai import *
from fastai.tabular import *
from tqdm import tqdm_notebook as tqdm

Tabular data should be in a Pandas `DataFrame`.

In [ ]:
path = Path('../Data/Cat-in-the-dat/') #untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
sample = pd.read_csv(path/'sample_submission.csv')

In [ ]:
path

In [ ]:
df.tail()

In [ ]:
test_df.head()

## Do some EDA

In [ ]:
df.id.nunique()

In [ ]:
df.bin_0.nunique(), df.bin_1.nunique(), df.bin_2.nunique(), df.bin_3.nunique(), df.bin_4.nunique()

In [ ]:
df.nom_0.nunique(), df.nom_1.nunique(), df.nom_2.nunique(), df.nom_3.nunique(), df.nom_4.nunique(), df.nom_5.nunique(), df.nom_6.nunique(), df.nom_7.nunique(), df.nom_8.nunique(), df.nom_9.nunique(), 

In [ ]:
df.ord_0.nunique(), df.ord_1.nunique(), df.ord_2.nunique(), df.ord_3.nunique(), df.ord_4.nunique(), df.ord_5.nunique() 

In [ ]:
df.day.nunique(), df.month.nunique()

In [ ]:
df.columns

In [ ]:
dep_var = 'target'
cat_names = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'nom_0', 'nom_1',
             'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8',
             'nom_9','ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5',
             'day', 'month']
cont_names = ['id']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
df.bin_0.dtype

In [ ]:
embed = {}
for col in cat_names:
#     print(col)
    if df[col].nunique() < 50:
        embed[col] = 50

In [ ]:
embed

In [ ]:
len(cat_names)

In [ ]:
test = TabularList.from_df(test_df, path=path, cat_names=cat_names, cont_names=cont_names)#, procs=procs)

In [ ]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(590000,599999)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch(bs=2048))

In [ ]:
data.show_batch(rows=5)

In [ ]:
# learn = tabular_learner(data, layers=[300,200,100], metrics=accuracy)

learn = tabular_learner(data, layers=[300,200,100],emb_szs=embed, emb_drop=0.3,ps=0.3, metrics=accuracy)

In [ ]:
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
data.batch_size

In [ ]:
learn.fit(5, 1e-2)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 1e-05)

## Inference

In [ ]:
sample.head()

In [ ]:
row = df.iloc[0]

In [ ]:
DatasetType.Test

In [ ]:
pred = learn.get_preds(DatasetType.Test)

In [ ]:
pred[0]

In [ ]:
pred[0]

In [ ]:
def get_submision(preds):
    labelled_preds = []
    pred11 = preds
    for pred in tqdm(pred11):
        labelled_preds.append(int(np.argmax(pred)))
    
    sample['target'] = labelled_preds
    
    return sample

In [ ]:
submission = get_submision(pred[0])

In [ ]:
submission.reset_index(drop=True)

In [ ]:
submission.to_csv('./submission_2.csv',index=False)